# KNN Reccomendation Algorithm

I decided to use the PyNNDescent, which is based on the NN Descent algorithm. I decided to use an approximate nearest neighbors instead of true KNN as I calculated the time to find the predictions on the full data set to be around 2.5 hours. This wasn't feasable as lots of iterations needed to be ran. I used NN Descent as it is a scalable algorithm with relatively low overhead, and had a highly recommended python library to go along with it. I would have used HNSW, however the memory overhead had me worried as memory consumption was already a significant bottleneck.

With NN Descent the total time to fit and predict on the full dataset is about 10 minutes, which is much more managable.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, vstack
from scipy.stats import pearsonr
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfTransformer
from math import ceil, floor, sqrt
from average import AverageRating
from tqdm import tqdm
from pynndescent.pynndescent_ import PyNNDescentTransformer

#### Importing Data

In [2]:
def load_data(sample):
    folder_path = "C:\\Users\\holcombea\\grad\\Recommendation Systems\\movie-lens"
    ratings_path = folder_path + "\\ratings.csv"

    if sample:
        ratings_path = "C:\\Users\\holcombea\\grad\\Recommendation Systems\\KNN\\sampe_data.csv"
        pass
    ratings = pd.read_csv(ratings_path)
    if not sample:
        original_movie_ids = set(ratings['movieId'])
        movie_id_map = {original: new for new, original in enumerate(original_movie_ids)}
        ratings['movieId'] = ratings['movieId'].map(movie_id_map)
    return ratings

In [3]:
ratings = load_data(sample=False)
data = csr_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))

In [4]:
print(data.shape)

(162542, 59047)


### KNN Class

In [5]:
class KNNRating:
    def __init__(self, k, metric: str):
        self.k = k
        self.metric = metric
        self.neighbors = PyNNDescentTransformer(n_neighbors=self.k, metric=self.metric)
        self.training_ratings = None


    def fit(self, training_ratings):
        max_k = training_ratings.shape[0]
        if self.k > max_k:
            print(f"k({self.k}) is greater than total samples, changing k to the number of samples({max_k})")
            self.k = max_k
        self.neighbors = PyNNDescentTransformer(n_neighbors=self.k, metric=self.metric)
        self.neighbors.fit(training_ratings)
        self.training_ratings = training_ratings
        

    def predict(self, user_ratings):
        if self.training_ratings is None:
            raise Exception("Must fit before predicting")
        distances = self.neighbors.transform(user_ratings)
        all_distances = distances.nonzero()[1]
        neighbor_indices = np.array_split(all_distances, ceil(all_distances.shape[0]/self.k))
        averageRating = AverageRating()
        predictions = list()
        for item in neighbor_indices:
            averageRating.fit(self.training_ratings[item])
            predictions.append(averageRating.predictions)
        return vstack(predictions)

## Evaluation

In [6]:
class DataPrep:
    """
    Class for splitting data and applying idf transformation.
    """
    def __init__(self, data, train_size=0.8, idf=False, norm=None):
        self.raw_data = data
        self.train_size = train_size
        self.train = None
        self.test = None
        self.seen = None
        self.unseen = None
        if idf:
            idf_transformer = TfidfTransformer(norm=norm, use_idf=True)
            self.raw_data = idf_transformer.fit_transform(self.raw_data)

    def split(self):
        """
        Splits data into train and test. Also splits test data into seen and unseen groups.
        """
        self.train, self.test = DataPrep.train_test_split(self.raw_data, train_size=self.train_size)
        self.seen, self.unseen = DataPrep.user_split(self.test)
        return self.train, self.seen, self.unseen
    
    @staticmethod
    def train_test_split(data, train_size):
        train, test = sklearn_train_test_split(data, train_size=train_size)
        return train, test

    @staticmethod
    def user_split(user_item_matrix: csr_matrix, split=0.8):
        seen_data = np.array([])
        seen_indices = np.array([])
        seen_indptr = np.array([0])

        unseen_data = np.array([])
        unseen_indices = np.array([])
        unseen_indptr = np.array([0])

        for i in range(len(user_item_matrix.indptr.copy()) - 1):
            row_start = user_item_matrix.indptr[i]
            row_end   = user_item_matrix.indptr[i+1] 
            sample_size = floor((row_end - row_start) * split)
            if sample_size == 0: #ensures something is in the test data
                sample_size += 1

            row_indices = user_item_matrix.indices[row_start: row_end]
            row_data = user_item_matrix.data[row_start: row_end]


            data_idx = np.arange(len(row_data))
            seen_idx = np.random.choice(data_idx, size=sample_size, replace=False)
            unseen_idx = np.setdiff1d(data_idx, seen_idx)
            
            #appending data to matrices
            seen_data = np.append(seen_data, row_data[seen_idx])
            unseen_data = np.append(unseen_data, row_data[unseen_idx])

            #appending indices
            seen_indices = np.append(seen_indices, row_indices[seen_idx])
            unseen_indices = np.append(unseen_indices, row_indices[unseen_idx])

            seen_indptr = np.append(seen_indptr, [seen_indptr[-1] + len(seen_idx)])
            unseen_indptr = np.append(unseen_indptr, [unseen_indptr[-1] + len(unseen_idx)])
        return csr_matrix((seen_data, seen_indices, seen_indptr), dtype=np.float32), csr_matrix((unseen_data, unseen_indices, unseen_indptr),dtype=np.float32)

In [7]:
def batch_predict(train, seen, unseen, k, metric,batch_size):
    model = KNNRating(k=k, metric=metric)
    model.fit(train)

    batched_input = list()
    if batch_size > seen.shape[0]:
        batched_input.append(seen)
    else:
        for i in range(0, seen.shape[0], batch_size):
            batched_input.append(seen[i:i+batch_size])
    predictions = list()
    for batch in tqdm(batched_input):
        predictions.append(model.predict(batch))
    y_pred = vstack(predictions)
    return y_pred, unseen

In [8]:
def score(predicted, true, rmse=True, pearson=True, sparsity=True, set_diff=True, debug=True):
    nonzeros = true.nonzero()
    y_true = np.array(true[nonzeros], dtype=np.float32)[0]
    y_pred = np.array(predicted[nonzeros], dtype=np.float32)[0]
    outputs = dict()
    if rmse:
        rmse = sqrt(mean_squared_error(y_true, y_pred))
        outputs['rmse'] = rmse
        if debug:
            print(f"rmse: {rmse}")
    if pearson:
        r_squared = pearsonr(y_true, y_pred)[1] ** 2
        outputs['r_squared'] = r_squared
        if debug:
            print(f"r2 score: {r_squared}")
    if set_diff:
        #batching set diff
        batch_size=400
        pred_batched = list()
        true_batched = list()
        for i in range(0, true.shape[0], batch_size):
            pred_batched.append(predicted[i:i+batch_size])
            true_batched.append(true[i:i+batch_size])
        pair_in_both = 0
        pair_in_true = 0
        for pred_batch, true_batch in tqdm(zip(pred_batched, true_batched)):
            pred_users, pred_movies = pred_batch.nonzero()
            pred_pairs = set(zip(pred_users,pred_movies))
            true_users, true_movies = true_batch.nonzero()
            true_pairs = set(zip(true_users, true_movies))

            pair_in_both += len(true_pairs.intersection(pred_pairs))
            pair_in_true += len(true_pairs)

        non_zero_ratings = pair_in_both / pair_in_true
        outputs['non_zero_ratings'] = non_zero_ratings
        if debug:
            print(f"Fraction of user-movie pairs with non-zero predicted ratings: {non_zero_ratings}")
    if sparsity:
        sparsity = predicted.getnnz() / (predicted.shape[0] * predicted.shape[1])
        outputs['sparsity'] = predicted.getnnz() / (predicted.shape[0] * predicted.shape[1])
        if debug:
            print(f"Sparsity: {sparsity}")
    return outputs

In [9]:
prep = DataPrep(data, idf=True, norm=None)
train, seen, unseen = prep.split()

In [10]:
def grid_search(train, seen, unseen, idf, norm, batch_size=100):
    ks = [5,10,25,50,100]
    metrics = ['euclidean','cosine']

    for k in ks:
        for metric in metrics:
            y_pred, y_true = batch_predict(train, seen, unseen, k=k, metric=metric, batch_size=batch_size)
            output = score(y_pred, y_true, set_diff=False, debug=False)
            output_str = f"k: {k}, metric: {metric}, idf: {idf}, norm: {norm}\n\trmse: {output['rmse']}\n\tr^2: {output['r_squared']}\n\tsparsity: {output['sparsity']}\n"
            with open("output.txt", "a") as f:
                f.write(output_str)

In [11]:
grid_search(train,seen,unseen,idf=True,norm=None)

  0%|          | 0/326 [00:00<?, ?it/s]c:\Users\holcombea\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
  0%|          | 0/326 [00:00<?, ?it/s]c:\Users\holcombea\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
  0%|          | 0/326 [00:00<?, ?it/s]c:\Users\holcombea\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
  0%|          | 0/326 [00:00<?, ?it/s]c:\Users\holcombea\AppData\Local\Programs\Python\Python311\Lib\site-packages\sc

In [ ]:
train

<130033x59047 sparse matrix of type '<class 'numpy.float64'>'
	with 19995357 stored elements in Compressed Sparse Row format>

In [ ]:
#y_pred, y_true = batch_predict(train, seen, unseen, k=4, metric='euclidean', batch_size=2)

In [ ]:
#score_output = score(y_pred, y_true, set_diff=False)

# Table of Results

|Parameter Input| RMSE | $r^2$| Sparsity|
|---------------|------|------|---------|
|5, E, No-IDF   |3.093 |0.0   | 0.00212 |
|5, C, No-IDF   |2.048 |0.0   | 0.00853 |
|10,E, No-IDF   |2.897 |0.0   | 0.00349 |
|10,C, No-IDF   |1.760 |0.0   | 0.0116  |
|25,E, No-IDF   |2.640 |0.0   | 0.006   |
|25,C, No-IDF   |1.456 |0.0   | 0.0176  |
|50,E, No-IDF   |2.429 |0.0   | 0.0102  |
|50,C, No-IDF   |1.282 |0.0   | 0.0243  |
|100,E,No-IDF   |2.217 |0.0   | 0.0158  |
|100,C,No-IDF   |1.154 |0.0   | 0.0335  |